In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
# lib import 
import numpy as np
from numpy import inf
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf

TRAINING = True

In [3]:
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    
    return df[features]

In [14]:
if TRAINING:
    # load dataset 
    df_train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
    df_features = generate_features(df_train)
    
    # data is defaulty normalized
    X = np.float32(df_features.values)
    Y = np.float32(df_train['target'].values)

    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]

    index = np.arange(len(X)) #array for indexing

    max_value = np.finfo(X.dtype).max #max float value allowed by numpy, perhaps lower? to prevent NAN during fit
    min_value = np.finfo(X.dtype).min
#     X[X==inf] = max_value
#     X[~np.isfinite(X)] = min_value
    X[X==inf] = 3.40282e+14
    X[~np.isfinite(X)] = -3.40282e+14
    
    del df_train
    del df_features

In [16]:
print(X[0:50000, :])

[[ 0.0000000e+00  1.0000000e+00  3.1806028e+06 ... -1.0001880e+00
  -1.0001880e+00  1.3829787e-01]
 [ 0.0000000e+00 -1.0000000e+00  1.6660391e+05 ... -1.0001041e+00
  -1.0001041e+00  6.3461537e+00]
 [ 0.0000000e+00 -1.0000000e+00  3.0287988e+05 ... -1.0005974e+00
  -1.0005974e+00  4.9916247e-01]
 ...
 [ 4.1000000e+02  1.0000000e+00  8.1980519e+05 ...  6.2000000e+01
   7.9180326e+00  4.0163937e+00]
 [ 4.1000000e+02  1.0000000e+00  2.6698028e+05 ...  1.3531250e+01
   3.8437500e+00  3.8437500e+00]
 [ 4.1000000e+02  0.0000000e+00  0.0000000e+00 ...  3.3544305e-01
   3.3544305e-01  3.3544305e-01]]


In [21]:
models = []

# test to train ratio
N_fold = 5

def train(tuned_model):
    if TRAINING:
        tuned_model.fit(X[index%N_fold!=0][0:80000, : ], Y[index%N_fold!=0][0:80000], verbose=1)

#         model = tfdf.keras.GradientBoostedTreesModel()
#         model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
#                     eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
#                     verbose=10, 
#                     early_stopping_rounds=100
#                     )

tuner = tfdf.tuner.RandomSearch(num_trials=100, use_predefined_hps=True)
tuned_model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION, tuner=tuner, verbose=1)
train(tuned_model)
tuned_model.compile(metrics=["mae"])


Use /tmp/tmpt556fjd1 as temporary training directory


[WARNING 23-11-04 05:55:36.1621 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-11-04 05:55:36.1622 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-11-04 05:55:36.1622 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Reading training dataset...
Training dataset read in 0:00:00.951984. Found 40000 examples.
Training model...
Model trained in 0:00:48.400020
Compiling model...


[INFO 23-11-04 05:56:25.8002 UTC kernel.cc:1243] Loading model from path /tmp/tmpt556fjd1/model/ with prefix 1c703a9cf0f84e5a
[INFO 23-11-04 05:56:25.8006 UTC decision_forest.cc:660] Model loaded with 1 root(s), 5 node(s), and 6 input feature(s).
[INFO 23-11-04 05:56:25.8006 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-11-04 05:56:25.8006 UTC kernel.cc:1075] Use fast generic engine


Model compiled.


In [19]:
tuned_test_accuracy = tuned_model.evaluate([(X[index%N_fold==0][0:20000, : ], Y[index%N_fold==0][0:20000])], return_dict=True, verbose=0)["mae"]

ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 546, in test_function  *
        return step_function_trained(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 525, in run_step  *
        outputs = model.test_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1788, in test_step  **
        y_pred = self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file97doke_r.py", line 232, in tf__call
        ag__.if_stmt(ag__.ld(self)._semantics is None, if_body_8, else_body_8, get_state_11, set_state_11, ('do_return', 'retval_'), 2)
    File "/tmp/__autograph_generated_file97doke_r.py", line 43, in else_body_8
        normaliz

In [20]:
tuning_logs = tuned_model.make_inspector().tuning_logs()
tuning_logs[tuning_logs.best].iloc[0]

score                                                -8.490697
evaluation_time                                     223.470499
best                                                      True
split_axis                                      SPARSE_OBLIQUE
sparse_oblique_projection_density_factor                   4.0
sparse_oblique_normalization                STANDARD_DEVIATION
sparse_oblique_weights                              CONTINUOUS
categorical_algorithm                                     CART
growing_strategy                                         LOCAL
max_depth                                                    3
sampling_method                                         RANDOM
subsample                                                  0.8
shrinkage                                                 0.05
min_examples                                                 7
num_candidate_attributes_ratio                             1.0
Name: 0, dtype: object

In [ ]:
'''
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()
'''

In [ ]:
'''
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    sample_prediction['target'] = model.predict(test)
    env.predict(sample_prediction)
    counter += 1
'''